In [84]:
#importing the required modules
import os
import math
import numpy as np
import pandas as pd

In [2]:
home_dir=os.path.join(r'Z:/balint/numt/')

In [3]:
master_array = pd.read_csv(home_dir+'Michael/data/numt_master_array.csv')
master_array.head()

,score,eg2_value,e_value,g_id,g_start,mt_start,g_length,mt_length,g_strand,mt_strand,g_size,g_sequence,mt_sequence,g_up_flanking,g_down_flanking,"mt_up_flanking (if strand is negative, the coordinates are corrected)","mt_down_flanking (if strand is negative, the coordinates are corrected)",ensembl_gene_id,ensembl_description
0,17245,0.0,0.0,MT,0,0,17245,17245,+,+,17245,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,NaN,NaN,TACAAATTGTATAATATTTGGACTAAACCCCCTTTCCCACCCCAAG...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,ENSOCUG00000029081,NaN
1,3398,0.0,0.0,1,114731432,6976,4680,4681,+,+,194850757,taGTAAAATTATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,TAGTAAAACCATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,ACAAAAGACTCTGAATAGCCAAAGCGATCCTGAACAAGAAAAATCA...,TATATATATTTATATATATATGTATATATGTACATATATATAACTA...,CCTTCGCCTCAAAACGAGAAGTAGAGACTATTGAACTAACAACCAC...,AACTGCTAATTCATGCTCCCATGTCTAACCCCATGGCTTTCTCAAC...,ENSOCUG00000006104,cilia and flagella associated protein 300 [Sou...
2,3120,0.0,0.0,GL019308,67160,883,4892,4945,+,+,88228,CAAGGCCATGAAGCATGCACACACTGCCCATCGCTCTCCTCAAACA...,CAAGGCCATGAAGCACGCACACACCGCCCGTCACCCTCCTCAAGTG...,GATAAACCCCGATAAACTTTACCACTCTTTGCCAACTCAGCCTCTA...,ATGAAGGGGCAAAGAGTAATTTTACAGTCGAGAGGGCTAGCAGTCA...,TACCGCCATCTTCAGCGAACCCTAAAAAGGAGCAAAAGTAAGCTCA...,TAGGGGCTATTAACTTTATTACAACTATTATTAATATGAAAGCCCC...,ENSOCUG00000031196,NaN
3,1888,0.0,0.0,GL019308,62527,12805,3439,3462,+,+,88228,AATTATTCATAGCCTCAATGATGAACAAGACATCCAAAAAACAGGA...,AATTATTCACAGCCTcaaTGATGAACAGGATATCCGAAAGATAGGC...,GTAGGAATTATATCCTTCCTCCTCATCGGCTGATGGCATGGCCAAG...,ACGTATACATGAGTTGTTCTTACATAATTGAATATACCTGCAATGC...,TGCCTAGGCGCAATCACTACCTTATTCACAGCCCTATGTGCCCTCA...,GTACACCCGTACGCACGCACGTACACCCGTACGCACGCACGTACAC...,ENSOCUG00000037635,NaN
4,1824,0.0,0.0,AAGW02081594,14016,6061,3040,3073,+,+,24576,TTCTTTGGACACCCTGAGGTATATATTCTTATCCTTCCAGGATTCG...,TTTTTCGGGCACCCCGAAGTATATATTCTTATTCTTCCAGGATTTG...,GACATGTCAAAGGAACGCAGAAGCCAGCTTGAAAAAGTCCTCAACA...,TGAAATAGCTATCACTATTCAGGTATGCCACTATTCTTCTAGGCAT...,ATATGAAAGCCCCTGCAATATCTCAATATCAAACCCCCTTATTCGT...,ATTCTCCTAGGTATCTATTTTACCCTACTTCAAGCATCGGAGTATT...,NaN,NaN


In [16]:
#function for writing outputs
def write_output(sequence, outfile):
    global seq_counter
    outfile.write('>'+str(seq_counter)+'\n')
    outfile.write(sequence+'\n')
    seq_counter+=1

In [51]:
#writing 5 bp upstream and downstream numt sequences into a fasta file
numt_ends=master_array['g_sequence'].apply(lambda sequence: sequence[:5].upper()+sequence[-5:].upper())
numt_ends.index=np.arange(0,len(numt_ends))
seq_counter=0
with open(home_dir+'seqlogo_numt_ends.fasta','w')as outfile:
    numt_ends.apply(write_output, args=(outfile,))

In [53]:
#get vector for aligned sequences
with open(home_dir+'numt_ends_alignment.fasta')as infile:
    content=pd.Series(infile.readlines())
    sequence_mask=content.apply(lambda line: '>' not in line)
    sequences=content[sequence_mask].apply(lambda line: line[:-1] if '\n' in line else line)
    sequences.index=np.arange(0,len(sequences))

In [132]:
#function for filling pfm
def pfm_filler(sequence, pfm):
    for index, nucleotide in enumerate(sequence):
        if nucleotide!='-' and nucleotide!='N':
            pfm.loc[nucleotide][index]+=1

In [173]:
ends_pfm=pd.DataFrame(0, columns=np.arange(0,len(sequences[0])),
                      index=['A','C','G','T'])
sequences.apply(pfm_filler, args=(ends_pfm,))
ends_pfm=ends_pfm.replace(0,1)
ends_pfm

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
A,4,3,42,23,40,51,24,44,44,41,44,21,25,36,1,2,7
C,3,1,3,15,25,29,30,26,29,7,16,8,44,19,5,2,1
G,18,4,22,20,14,13,27,31,6,42,16,45,10,8,7,2,1
T,1,43,16,22,33,27,65,51,73,59,53,45,14,10,15,6,1


In [174]:
ends_pwm=np.log2(ends_pfm/0.25)
ends_pwm

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
A,4.000000,3.584963,7.392317,6.523562,7.321928,7.672425,6.584963,7.459432,7.459432,7.357552,7.459432,6.392317,6.643856,7.169925,2.000000,3.000000,4.807355
C,3.584963,2.000000,3.584963,5.906891,6.643856,6.857981,6.906891,6.700440,6.857981,4.807355,6.000000,5.000000,7.459432,6.247928,4.321928,3.000000,2.000000
G,6.169925,4.000000,6.459432,6.321928,5.807355,5.700440,6.754888,6.954196,4.584963,7.392317,6.000000,7.491853,5.321928,5.000000,4.807355,3.000000,2.000000
T,2.000000,7.426265,6.000000,6.459432,7.044394,6.754888,8.022368,7.672425,8.189825,7.882643,7.727920,7.491853,5.807355,5.321928,5.906891,4.584963,2.000000


In [175]:
#function for flanking preparation
def flanking_prep(row):
    global flankings
    if type(row['g_up_flanking'])!=str or type(row['g_down_flanking'])!=str:
        pass
    else:
        flankings.append(row['g_up_flanking'][-5:]+row['g_down_flanking'][:5])

In [176]:
flankings=[]
master_array.apply(flanking_prep, axis=1)
flankings=pd.Series(flankings)
flankings

0      ATATATATAT
1      AAGGCATGAA
2      CCACTACGTA
3      CCCTGTGAAA
4      GGCCAGGCTG
          ...    
143    GTATTAGTTA
144    GTCGTGTACG
145    TTACTGTTTA
146    TCGTTGCATG
147    AGACTACTAC
Length: 148, dtype: object

In [126]:
with open(home_dir+'seqlogo_numt_flankings.fasta','w')as outfile:
    flankings.apply(write_output, args=(outfile,))

In [129]:
#get vector for aligned sequences
with open(home_dir+'numt_flankings_alignment.fasta')as infile:
    content=pd.Series(infile.readlines())
    sequence_mask=content.apply(lambda line: '>' not in line)
    aligned_flankings=content[sequence_mask].apply(lambda line: line[:-1] if '\n' in line else line)
    aligned_flankings.index=np.arange(0,len(aligned_flankings))

In [177]:
flankings_pfm=pd.DataFrame(0, columns=np.arange(0,len(aligned_flankings[0])),
                      index=['A','C','G','T'])
aligned_flankings.apply(pfm_filler, args=(flankings_pfm,))
flankings_pfm=flankings_pfm.replace(0,1)
flankings_pfm

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
A,3,3,14,24,9,58,30,25,67,37,28,50,21,2,19,6,11,1
C,1,1,6,10,37,22,40,18,21,32,14,6,13,18,12,12,1,1
G,1,5,5,25,17,24,44,34,32,24,21,19,16,25,7,2,1,1
T,1,1,16,12,35,21,25,67,26,51,65,42,55,32,14,5,1,1


In [178]:
flankings_pwm=np.log2(flankings_pfm/0.25)
flankings_pwm

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
A,3.584963,3.584963,5.807355,6.584963,5.169925,7.857981,6.906891,6.643856,8.066089,7.209453,6.807355,7.643856,6.392317,3.000000,6.247928,4.584963,5.459432,2.0
C,2.000000,2.000000,4.584963,5.321928,7.209453,6.459432,7.321928,6.169925,6.392317,7.000000,5.807355,4.584963,5.700440,6.169925,5.584963,5.584963,2.000000,2.0
G,2.000000,4.321928,4.321928,6.643856,6.087463,6.584963,7.459432,7.087463,7.000000,6.584963,6.392317,6.247928,6.000000,6.643856,4.807355,3.000000,2.000000,2.0
T,2.000000,2.000000,6.000000,5.584963,7.129283,6.392317,6.643856,8.066089,6.700440,7.672425,8.022368,7.392317,7.781360,7.000000,5.807355,4.321928,2.000000,2.0


In [180]:
ends_pfm.to_csv(home_dir+'ends_pfm.csv')
ends_pwm.to_csv(home_dir+'ends_pwm.csv')
flankings_pfm.to_csv(home_dir+'flankings_pfm.csv')
flankings_pwm.to_csv(home_dir+'flankings_pwm.csv')